In [10]:
import re
import os
from time import gmtime, strftime
from datetime import datetime, timedelta
import unicodedata

# Importing libraries you need to install
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import requests
import bs4 as bs
from lxml import html
from tqdm import tqdm
import threading
import time as t_time
from ratelimit import limits

In [11]:
!pip install ratelimit

### read and have a look at the data

In [12]:
#read the ticker library of all the tikers into ticker_library
ticker_library = pd.read_csv('tickers.csv')

#read the sp500 components into ticker_selected, 'name' is the company name and ticker is company's ticker
ticker_selected = pd.read_csv('SP500_component_stocks.csv',header = None)
ticker_selected.columns = ['name','ticker']

#display a sample of ticker_selected
display(ticker_selected)


/opt/miniconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3145: DtypeWarning: Columns (15,16) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,name,ticker
0,Agilent Technologies Inc.,A
1,American Airlines Group Inc.,AAL
2,Advance Auto Parts Inc.,AAP
3,Apple Inc.,AAPL
4,AbbVie Inc.,ABBV
...,...,...
500,Xylem Inc.,XYL
501,Yum! Brands Inc.,YUM
502,Zimmer Biomet Holdings Inc.,ZBH
503,Zions Bancorporation N.A.,ZION


### Prepare a cik&ticker dataframe

In [13]:
#build a ticker_cik_df dataframe to store ticker and its cik number
ticker_cik_df = pd.DataFrame()

#store all the tickers in a ticker_list
ticker_list = ticker_selected.ticker.unique()

#build a list cik_list for cik
cik_list = []

for ticker in ticker_list:    
    try:
        #for a given ticker, find its cik number through th ticker library
        cik_list.append(list(ticker_library[ticker_library.ticker == ticker].secfilings)[0][-10:])
        
    except:
        #if could not find cik, give it a empty cik
        cik_list.append('')

#write cik_list and ticker_list to the dataframe ticker_cik_df
ticker_cik_df['cik'] = cik_list
ticker_cik_df['ticker'] = ticker_list

#delete the tickers with empty cik number
ticker_cik_df = ticker_cik_df[ticker_cik_df['cik'] != '']

#display a sample of ticker_cik_df
ticker_cik_df

,cik,ticker
0,0001090872,A
1,0000006201,AAL
2,0001158449,AAP
3,0000320193,AAPL
4,0001551152,ABBV
...,...,...
500,0001524472,XYL
501,0001041061,YUM
502,0001136869,ZBH
503,0000109380,ZION


In [14]:
listtickers = ['AMZN','BBY','BKNG','MCD','EBAY','F','HD','TGT','WHR','JPM','SIVB','CFG','C','ALL','IVZ','ETFC','MET','PFG','CBOE',
              'CTL','IPG','VIAC','NFLX','CHTR','FB','TWTR','NWSA','FOXA','AMD','INTC','AAPL','LRCX','MSFT','NLOK','CTSH','ADS',
              'WU','PAYC','ABT','CVS','PFE','JNJ','BIIB','INCY','HSIC','WAT','ALGN','EW']
listrows = []
for j in listtickers:
    row = ticker_cik_df[ticker_cik_df['ticker'] == j]
    listrows.append(row)

In [15]:
ticker_cik_sample = pd.DataFrame()
for i in range(len(listrows)):
    ticker_cik_sample = pd.concat([ticker_cik_sample,listrows[i]],ignore_index=False)
ticker_cik_sample

,cik,ticker
39,0001018724,AMZN
64,0000764478,BBY
71,0001075531,BKNG
304,0000063908,MCD
156,0001065088,EBAY
178,0000037996,F
222,0000354950,HD
435,0000027419,TGT
485,0000106640,WHR
266,0000019617,JPM


In [16]:
#For demostration purposes we will only scrape two companies
ticker_cik_sample = ticker_cik_df.sample(n=2,replace=False,random_state=0)
ticker_cik_sample

,cik,ticker
92,0000816284,CELG
257,0000049826,ITW


In [17]:
ticker_cik_sample

,cik,ticker
92,0000816284,CELG
257,0000049826,ITW


### import function to scrape 

In [18]:
def WriteLogFile(log_file_name, text):
    '''
    Helper function.
    Writes a log file with all notes and
    error messages from a scraping "session".
    
    Parameters
    ----------
    log_file_name : str
        Name of the log file (should be a .txt file).
    text : str
        Text to write to the log file.
        
    Returns
    -------
    None.
    
    '''
    with open(log_file_name, "a") as log_file:
        log_file.write(text)

    return

In [19]:
def Scrape10K(browse_url_base, filing_url_base, doc_url_base, cik, log_file_name):
    
    '''
    Scrapes all 10-Ks and 10-K405s for a particular 
    CIK from EDGAR.
    
    Parameters
    ----------
    browse_url_base : str
        Base URL for browsing EDGAR.
    filing_url_base : str
        Base URL for filings listings on EDGAR.
    doc_url_base : str
        Base URL for one filing's document tables
        page on EDGAR.
    cik : str
        Central Index Key.
    log_file_name : str
        Name of the log file (should be a .txt file).
        
    Returns
    -------
    None.
    
    '''
    
    # Check if we've already scraped this CIK
    try:
        os.mkdir(cik)
    except OSError:
        print("Already scraped CIK", cik)
        return
    
    # If we haven't, go into the directory for that CIK
    os.chdir(cik)
    
    #I avoid print here
    """
    print('Scraping CIK', cik)
    """
    
    # Request list of 10-K filings
    res = requests.get(browse_url_base % cik)
    
    # If the request failed, log the failure and exit
    if res.status_code != 200:
        os.chdir('..')
        os.rmdir(cik) # remove empty dir
        text = "Request failed with error code " + str(res.status_code) + \
               "\nFailed URL: " + (browse_url_base % cik) + '\n'
        WriteLogFile(log_file_name, text)
        return

    # If the request doesn't fail, continue...
    
    # Parse the response HTML using BeautifulSoup
    soup = bs.BeautifulSoup(res.text, "lxml")

    # Extract all tables from the response
    html_tables = soup.find_all('table')
    
    # Check that the table we're looking for exists
    # If it doesn't, exit
    if len(html_tables)<3:
        os.chdir('..')
        return
    
    # Parse the Filings table
    filings_table = pd.read_html(str(html_tables[2]), header=0)[0]
    filings_table['Filings'] = [str(x) for x in filings_table['Filings']]

    # Get only 10-K and 10-K405 document filings
    filings_table = filings_table[(filings_table['Filings'] == '10-K') | (filings_table['Filings'] == '10-K405')]

    # If filings table doesn't have any
    # 10-Ks or 10-K405s, exit
    if len(filings_table)==0:
        os.chdir('..')
        return
    
    # Get accession number for each 10-K and 10-K405 filing
    filings_table['Acc_No'] = [x.replace('\xa0',' ')
                               .split('Acc-no: ')[1]
                               .split(' ')[0] for x in filings_table['Description']]

    # Iterate through each filing and 
    # scrape the corresponding document...
    for index, row in filings_table.iterrows():
        
        # Get the accession number for the filing
        acc_no = str(row['Acc_No'])
        
        # Navigate to the page for the filing
        docs_page = requests.get(filing_url_base % (cik, acc_no))
        
        # If request fails, log the failure
        # and skip to the next filing
        if docs_page.status_code != 200:
            os.chdir('..')
            text = "Request failed with error code " + str(docs_page.status_code) + \
                   "\nFailed URL: " + (filing_url_base % (cik, acc_no)) + '\n'
            WriteLogFile(log_file_name, text)
            os.chdir(cik)
            continue

        # If request succeeds, keep going...
        
        # Parse the table of documents for the filing
        docs_page_soup = bs.BeautifulSoup(docs_page.text, 'lxml')
        docs_html_tables = docs_page_soup.find_all('table')
        if len(docs_html_tables)==0:
            continue
        docs_table = pd.read_html(str(docs_html_tables[0]), header=0)[0]
        docs_table['Type'] = [str(x) for x in docs_table['Type']]
        
        # Get the 10-K and 10-K405 entries for the filing
        docs_table = docs_table[(docs_table['Type'] == '10-K') | (docs_table['Type'] == '10-K405')]
        
        # If there aren't any 10-K or 10-K405 entries,
        # skip to the next filing
        if len(docs_table)==0:
            continue
        # If there are 10-K or 10-K405 entries,
        # grab the first document
        elif len(docs_table)>0:
            docs_table = docs_table.iloc[0]
        
        docname = docs_table['Document']
        
        # If that first entry is unavailable,
        # log the failure and exit
        if str(docname) == 'nan':
            os.chdir('..')
            text = 'File with CIK: %s and Acc_No: %s is unavailable' % (cik, acc_no) + '\n'
            WriteLogFile(log_file_name, text)
            os.chdir(cik)
            continue       
        
        # If it is available, continue...
        
        # Request the file
        file = requests.get(doc_url_base % (cik, acc_no.replace('-', ''), docname))
        
        # If the request fails, log the failure and exit
        if file.status_code != 200:
            os.chdir('..')
            text = "Request failed with error code " + str(file.status_code) + \
                   "\nFailed URL: " + (doc_url_base % (cik, acc_no.replace('-', ''), docname)) + '\n'
            WriteLogFile(log_file_name, text)
            os.chdir(cik)
            continue
        
        # If it succeeds, keep going...
        
        # Save the file in appropriate format
        """
        date = str(row['Filing Date'])
        filename = cik + '_' + date + '.txt'
        html_file = open(filename, 'a')
        html_file.write(file.text)
        html_file.close()
        """
        if '.txt' in docname:
            # Save text as TXT
            date = str(row['Filing Date'])
            filename = cik + '_' + date + '.txt'
            html_file = open(filename, 'a')
            html_file.write(file.text)
            html_file.close()
        else:
            # Save text as HTML
            date = str(row['Filing Date'])
            filename = cik + '_' + date + '.html'
            html_file = open(filename, 'a')
            html_file.write(file.text)
            html_file.close()
        
    # Move back to the main 10-K directory
    os.chdir('..')
        
    return

In [20]:
def Scrape10Q(browse_url_base, filing_url_base, doc_url_base, cik, log_file_name):
    
    '''
    Scrapes all 10-Qs for a particular CIK from EDGAR.
    
    Parameters
    ----------
    browse_url_base : str
        Base URL for browsing EDGAR.
    filing_url_base : str
        Base URL for filings listings on EDGAR.
    doc_url_base : str
        Base URL for one filing's document tables
        page on EDGAR.
    cik : str
        Central Index Key.
    log_file_name : str
        Name of the log file (should be a .txt file).
        
    Returns
    -------
    None.
    
    '''
    
    # Check if we've already scraped this CIK
    try:
        os.mkdir(cik)
    except OSError:
        print("Already scraped CIK", cik)
        return
    
    # If we haven't, go into the directory for that CIK
    os.chdir(cik)
    
    #I avoid print here
    """
    print('Scraping CIK', cik)
    """
    
    # Request list of 10-Q filings
    res = requests.get(browse_url_base % cik)
    
    # If the request failed, log the failure and exit
    if res.status_code != 200:
        os.chdir('..')
        os.rmdir(cik) # remove empty dir
        text = "Request failed with error code " + str(res.status_code) + \
               "\nFailed URL: " + (browse_url_base % cik) + '\n'
        WriteLogFile(log_file_name, text)
        return
    
    # If the request doesn't fail, continue...

    # Parse the response HTML using BeautifulSoup
    soup = bs.BeautifulSoup(res.text, "lxml")

    # Extract all tables from the response
    html_tables = soup.find_all('table')
    
    # Check that the table we're looking for exists
    # If it doesn't, exit
    if len(html_tables)<3:
        print("table too short")
        os.chdir('..')
        return
    
    # Parse the Filings table
    filings_table = pd.read_html(str(html_tables[2]), header=0)[0]
    filings_table['Filings'] = [str(x) for x in filings_table['Filings']]

    # Get only 10-Q document filings
    filings_table = filings_table[filings_table['Filings'] == '10-Q']

    # If filings table doesn't have any
    # 10-Ks or 10-K405s, exit
    if len(filings_table)==0:
        os.chdir('..')
        return
    
    # Get accession number for each 10-K and 10-K405 filing
    filings_table['Acc_No'] = [x.replace('\xa0',' ')
                               .split('Acc-no: ')[1]
                               .split(' ')[0] for x in filings_table['Description']]

    # Iterate through each filing and 
    # scrape the corresponding document...
    for index, row in filings_table.iterrows():
        
        # Get the accession number for the filing
        acc_no = str(row['Acc_No'])
        
        # Navigate to the page for the filing
        docs_page = requests.get(filing_url_base % (cik, acc_no))
        
        # If request fails, log the failure
        # and skip to the next filing    
        if docs_page.status_code != 200:
            os.chdir('..')
            text = "Request failed with error code " + str(docs_page.status_code) + \
                   "\nFailed URL: " + (filing_url_base % (cik, acc_no)) + '\n'
            WriteLogFile(log_file_name, text)
            os.chdir(cik)
            continue
            
        # If request succeeds, keep going...
        
        # Parse the table of documents for the filing
        docs_page_soup = bs.BeautifulSoup(docs_page.text, 'lxml')
        docs_html_tables = docs_page_soup.find_all('table')
        if len(docs_html_tables)==0:
            continue
        docs_table = pd.read_html(str(docs_html_tables[0]), header=0)[0]
        docs_table['Type'] = [str(x) for x in docs_table['Type']]
        
        # Get the 10-K and 10-K405 entries for the filing
        docs_table = docs_table[docs_table['Type'] == '10-Q']
        
        # If there aren't any 10-K or 10-K405 entries,
        # skip to the next filing
        if len(docs_table)==0:
            continue
        # If there are 10-K or 10-K405 entries,
        # grab the first document
        elif len(docs_table)>0:
            docs_table = docs_table.iloc[0]
        
        docname = docs_table['Document']
        
        # If that first entry is unavailable,
        # log the failure and exit
        if str(docname) == 'nan':
            os.chdir('..')
            text = 'File with CIK: %s and Acc_No: %s is unavailable' % (cik, acc_no) + '\n'
            WriteLogFile(log_file_name, text)
            os.chdir(cik)
            continue       
        
        # If it is available, continue...
        
        # Request the file
        file = requests.get(doc_url_base % (cik, acc_no.replace('-', ''), docname))
        
        # If the request fails, log the failure and exit
        if file.status_code != 200:
            os.chdir('..')
            text = "Request failed with error code " + str(file.status_code) + \
                   "\nFailed URL: " + (doc_url_base % (cik, acc_no.replace('-', ''), docname)) + '\n'
            WriteLogFile(log_file_name, text)
            os.chdir(cik)
            continue
            
        # If it succeeds, keep going...
        
        # Save the file in appropriate format
        """
        date = str(row['Filing Date'])
        filename = cik + '_' + date + '.txt'
        html_file = open(filename, 'a')
        html_file.write(file.text)
        html_file.close()
        """
        if '.txt' in docname:
            # Save text as TXT
            date = str(row['Filing Date'])
            filename = cik + '_' + date + '.txt'
            html_file = open(filename, 'a')
            html_file.write(file.text)
            html_file.close()
        else:
            # Save text as HTML
            date = str(row['Filing Date'])
            filename = cik + '_' + date + '.html'
            html_file = open(filename, 'a')
            html_file.write(file.text)
            html_file.close()
        
    # Move back to the main 10-Q directory
    os.chdir('..')
        
    return

### set up path

In [21]:
pathname_10k = './Scrape_data/10_k'
pathname_10q = './Scrape_data/10_q'

### scrape

In [22]:
# Run the function to scrape 10-Ks

# Define parameters
browse_url_base_10k = 'https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=%s&type=10-K'
filing_url_base_10k = 'http://www.sec.gov/Archives/edgar/data/%s/%s-index.html'
doc_url_base_10k = 'http://www.sec.gov/Archives/edgar/data/%s/%s/%s'

# Set correct directory
try:
    os.chdir(pathname_10k)
except Exception:
    os.makedirs(pathname_10k)
    os.chdir(pathname_10k)

# Initialize log file
# (log file name = the time we initiate scraping session)
time = strftime("%Y-%m-%d %Hh%Mm%Ss", gmtime())
log_file_name = 'log '+time+'.txt'
with open(log_file_name, 'a') as log_file:
    log_file.close()

# Iterate over CIKs and scrape 10-Ks
for cik in tqdm(ticker_cik_sample['cik']):
    Scrape10K(browse_url_base=browse_url_base_10k, 
          filing_url_base=filing_url_base_10k, 
          doc_url_base=doc_url_base_10k, 
          cik=cik,
          log_file_name=log_file_name)
    

#return to the main menu
os.chdir('..')
os.chdir('..')

100%|██████████| 2/2 [00:27<00:00, 13.63s/it]


In [23]:
# Run the function to scrape 10-Qs

# Define parameters
browse_url_base_10q = 'https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=%s&type=10-Q&count=1000'
filing_url_base_10q = 'http://www.sec.gov/Archives/edgar/data/%s/%s-index.html'
doc_url_base_10q = 'http://www.sec.gov/Archives/edgar/data/%s/%s/%s'

# Set correct directory (fill this out yourself!)
try:
    os.chdir(pathname_10q)
except Exception:
    os.makedirs(pathname_10q)
    os.chdir(pathname_10q)

# Initialize log file
# (log file name = the time we initiate scraping session)
time = strftime("%Y-%m-%d %Hh%Mm%Ss", gmtime())
log_file_name = 'log '+time+'.txt'
log_file = open(log_file_name, 'a')
log_file.close()

# Iterate over CIKs and scrape 10-Qs
for cik in tqdm(ticker_cik_sample['cik']):
    Scrape10Q(browse_url_base = browse_url_base_10q, 
          filing_url_base = filing_url_base_10q, 
          doc_url_base = doc_url_base_10q, 
          cik = cik,
          log_file_name = log_file_name)

#return to the main menu
os.chdir('..')
os.chdir('..')

100%|██████████| 2/2 [01:19<00:00, 39.97s/it]


### convert html files to pure txt: preparing function

In [24]:
def RemoveNumericalTables(soup):
    
    '''
    Removes tables with >15% numerical characters.
    
    Parameters
    ----------
    soup : BeautifulSoup object
        Parsed result from BeautifulSoup.
        
    Returns
    -------
    soup : BeautifulSoup object
        Parsed result from BeautifulSoup
        with numerical tables removed.
        
    '''
    
    # Determines percentage of numerical characters
    # in a table
    def GetDigitPercentage(tablestring):
        if len(tablestring)>0.0:
            numbers = sum([char.isdigit() for char in tablestring])
            length = len(tablestring)
            return numbers/length
        else:
            return 1
    
    # Evaluates numerical character % for each table
    # and removes the table if the percentage is > 15%
    [x.extract() for x in soup.find_all('table') if GetDigitPercentage(x.get_text())>0.15]
    
    return soup

In [25]:
def RemoveTags(soup):
    
    '''
    Drops HTML tags, newlines and unicode text from
    filing text.
    
    Parameters
    ----------
    soup : BeautifulSoup object
        Parsed result from BeautifulSoup.
        
    Returns
    -------
    text : str
        Filing text.
        
    '''
    
    # Remove HTML tags with get_text
    text = soup.get_text()
    
    # Remove newline characters
    text = text.replace('\n', ' ')
    
    '''
    Adding a replace to convert all excecute space to one space 
    '''
    text = re.sub(r'\s', ' ', text)
    
    # Replace unicode characters with their
    # "normal" representations
    text = unicodedata.normalize('NFKD', text)
    
    return text

In [26]:
def ConvertHTML(cik):
    
    '''
    Removes numerical tables, HTML tags,
    newlines, unicode text, and XBRL tables.
    
    Parameters
    ----------
    cik : str
        Central Index Key used to scrape files.
    
    Returns
    -------
    None.
    
    '''
    
    # Look for files scraped for that CIK
    try: 
        os.chdir(cik)
    # ...if we didn't scrape any files for that CIK, exit
    except FileNotFoundError:
        print("Could not find directory for CIK", cik)
        return
    """ avoid print"""    
    #print("Parsing CIK %s..." % cik)
    
    parsed = False # flag to tell if we've parsed anything
    
    # Try to make a new directory within the CIK directory
    # to store the text representations of the filings
    try:
        os.mkdir('rawtext')
    # If it already exists, continue
    # We can't exit at this point because we might be
    # partially through parsing text files, so we need to continue
    except OSError:
        pass
    
    # Get list of scraped files
    # excluding hidden files and directories
    file_list = [fname for fname in os.listdir() if not (fname.startswith('.') | os.path.isdir(fname))]
    
    # Iterate over scraped files and clean
    for filename in file_list:
            
        # Check if file has already been cleaned
        new_filename = filename.replace('.html', '.txt')
        text_file_list = os.listdir('rawtext')
        if new_filename in text_file_list:
            continue
        
        # If it hasn't been cleaned already, keep going...
        
        # Clean file
        with open(filename, 'r',encoding='utf-8') as file:
            parsed = True
            soup = bs.BeautifulSoup(file.read(), "lxml")
            soup = RemoveNumericalTables(soup)
            
            #add my delete reference function here
            #soup = delete_reference(soup)
            
            text = RemoveTags(soup)
            with open('rawtext/'+new_filename, 'w',encoding='utf-8') as newfile:
                newfile.write(text)
    
    # If all files in the CIK directory have been parsed
    # then log that
    if parsed==False:
        print("Already parsed CIK", cik)
    
    os.chdir('..')
    return

In [27]:
'''
In the documents, there are some parts not in the Management Discussion but contain reference to this part,like:
'For a discussion of our contractual obligations, see “Item 7. Management’s Discussion and Analysis of Financial Condition and Results of Operations,”'' 

That is why I need a function to delete all the reference parts of MD&A.

'''

#compose a function to delete the references
def delete_reference(soup):
    
    #define string reference to the management's discussion parts
    management_str = "Financial Condition and Results of Operations"
                    
    #define string reference to the financial statement parts, which is what we need for parsing
    financial_str = "and Supplementary Data"
    
    #given the none reference length for comparision
    length_1 = len('Management’s Discussion and Analysis of Financial Condition and Results of Operations')
    
    #given the none reference length for comparision
    length_2 = len('Financial Statements and Supplementary Data')
    
    #find all the parts contains this reference
    for part in soup.find_all(text = re.compile(management_str)):
        
        #if the lenth is much greater than the original string length, then it's a reference, and we should delete it
        if len(part) > length_1:
            
            #we should extract these text
            part.extract()
            
    #the same should be operated to the financial statements
    for part in soup.find_all(text = re.compile(financial_str)):
        if len(part) > length_2:
            part.extract()
    
    #return soup for futher parsing
    return soup


### convert html to txt

In [28]:
# For 10-Ks...
# -*- coding: utf-8 -*-
os.chdir(pathname_10k)

# Iterate over CIKs and clean HTML filings
for cik in tqdm(ticker_cik_sample['cik']):
    ConvertHTML(cik)
os.chdir('..')
os.chdir('..')

100%|██████████| 2/2 [00:29<00:00, 14.88s/it]


In [29]:
# For 10-Qs...

os.chdir(pathname_10q)

# Iterate over CIKs and clean HTML filings
for cik in tqdm(ticker_cik_sample['cik']):
    ConvertHTML(cik)
os.chdir('..')
os.chdir('..')

100%|██████████| 2/2 [00:50<00:00, 25.08s/it]


In [30]:
os.getcwd()

'/Users/silviaruiz/Documents/Code/Github'

In [31]:
os.chdir('..')